In [5]:
import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
my_collection = mydb['reviews']

doc_complete = [review_doc["buyerTranslationFeedback"] 
                for review_doc in my_collection.find() 
                if "buyerTranslationFeedback" in review_doc.keys() ]

print("count of reviews: "+ str(len(doc_complete)))



count of reviews: 6644


In [15]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    #stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    tokens = word_tokenize(doc.lower())
    normalized = " ".join(lemma.lemmatize(token) for token in tokens if token not in stop and token not in exclude)
    return normalized

doc_clean = [clean(doc) for doc in doc_complete]   
print(doc_clean[:100])

['msk', 'region', 'week', 'use', 'phone', 'thank', 'phone', 'hope', 'phone', 'recommend', 'phone', 'голдвея', 'recommend', 'seller', 'delivery', 'day', 'region', 'excellent', 'purchase', 'band', 'work', 'phone', 'payment', 'recommend', 'case', 'material', 'delicate', 'phone', 'region', 'week', 'purchase', 'thank', 'phone', 'seller', 'order', 'figment', 'work', 'week', 'gift', 'wrap', 'order', 'phone', 'seller', 'time', 'phone', 'seller', 'family', 'delivery', 'week', 'quality', 'gift', 'seller', 'film', 'screen', 'trifle', 'packaging', 'nothing', 'gift', 'phone', 'fire', 'everything', 'seller', 'charger', 'type', 'c', 'type', 'everything', 'order', 'phone', 'book', 'seller', 'everything', 'day', 'packaging', 'quality', 'everything', 'ok', 'day', 'phone', 'chelny', 'day', 'phone', 'everything', 'delivery', 'day', 'thank', 'phone', 'impression', 'seller', 'honest', 'moment', 'claim', 'phone', 'continue', 'week', 'time', 'everything', 'seller', 'order', 'excellent']


In [16]:
import nltk

full_text = " ".join([review for review in doc_clean])

tokens = nltk.word_tokenize(full_text)

tagged = nltk.pos_tag(tokens)
print(tagged[:100])

[('msk', 'JJ'), ('region', 'NN'), ('week', 'NN'), ('use', 'NN'), ('phone', 'NN'), ('thank', 'NN'), ('phone', 'NN'), ('hope', 'VBP'), ('phone', 'NN'), ('recommend', 'NN'), ('phone', 'NN'), ('голдвея', 'NNP'), ('recommend', 'VBP'), ('seller', 'NN'), ('delivery', 'NN'), ('day', 'NN'), ('region', 'NN'), ('excellent', 'JJ'), ('purchase', 'NN'), ('band', 'NN'), ('work', 'NN'), ('phone', 'NN'), ('payment', 'NN'), ('recommend', 'NN'), ('case', 'NN'), ('material', 'JJ'), ('delicate', 'NN'), ('phone', 'NN'), ('region', 'NN'), ('week', 'NN'), ('purchase', 'NN'), ('thank', 'VBD'), ('phone', 'NN'), ('seller', 'NN'), ('order', 'NN'), ('figment', 'NN'), ('work', 'NN'), ('week', 'NN'), ('gift', 'NN'), ('wrap', 'NN'), ('order', 'NN'), ('phone', 'NN'), ('seller', 'NN'), ('time', 'NN'), ('phone', 'NN'), ('seller', 'VBD'), ('family', 'NN'), ('delivery', 'NN'), ('week', 'NN'), ('quality', 'NN'), ('gift', 'NN'), ('seller', 'NN'), ('film', 'NN'), ('screen', 'NN'), ('trifle', 'NN'), ('packaging', 'VBG'), ('no

In [17]:
#nouns = [token for token, pos in tagged if pos.startswith('N')]
#print(nouns)
  	
grammar = r"""
  NOUN_PHRASE:  {<CD|IN|DT|JJ|PP|RB|\$>*<N.*>+<V.*>*<CD|IN|DT|JJ|PP|RB|\$>*<N.*>*}  
"""
cp = nltk.RegexpParser(grammar)
noun_phrase_tagged = cp.parse(tagged)
print(noun_phrase_tagged[:100])
#doc_complete = nouns

[Tree('NOUN_PHRASE', [('msk', 'JJ'), ('region', 'NN'), ('week', 'NN'), ('use', 'NN'), ('phone', 'NN'), ('thank', 'NN'), ('phone', 'NN'), ('hope', 'VBP'), ('phone', 'NN'), ('recommend', 'NN'), ('phone', 'NN'), ('голдвея', 'NNP')]), ('recommend', 'VBP'), Tree('NOUN_PHRASE', [('seller', 'NN'), ('delivery', 'NN'), ('day', 'NN'), ('region', 'NN'), ('excellent', 'JJ'), ('purchase', 'NN'), ('band', 'NN'), ('work', 'NN'), ('phone', 'NN'), ('payment', 'NN'), ('recommend', 'NN'), ('case', 'NN')]), Tree('NOUN_PHRASE', [('material', 'JJ'), ('delicate', 'NN'), ('phone', 'NN'), ('region', 'NN'), ('week', 'NN'), ('purchase', 'NN'), ('thank', 'VBD'), ('phone', 'NN'), ('seller', 'NN'), ('order', 'NN'), ('figment', 'NN'), ('work', 'NN'), ('week', 'NN'), ('gift', 'NN'), ('wrap', 'NN'), ('order', 'NN'), ('phone', 'NN'), ('seller', 'NN'), ('time', 'NN'), ('phone', 'NN')]), ('seller', 'VBD'), Tree('NOUN_PHRASE', [('family', 'NN'), ('delivery', 'NN'), ('week', 'NN'), ('quality', 'NN'), ('gift', 'NN'), ('sell

In [18]:
def traverse(t,noun_phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            [noun_phrases.append(word) for (word,pos) in t.leaves() if pos.startswith("N")]
            #print(statement)
            #noun_phrases = noun_phrases.append(noun_phrase)
        # Now we know that t.node is defined
        #print('(', t.label(), end=" ")
        for child in t:
            traverse(child,noun_phrases)
        #print(')', end=" ")
    return noun_phrases

noun_phrases=[]
nouns=traverse(noun_phrase_tagged,noun_phrases)
print(nouns[:100])
doc_complete = nouns

['region', 'week', 'use', 'phone', 'thank', 'phone', 'phone', 'recommend', 'phone', 'голдвея', 'seller', 'delivery', 'day', 'region', 'purchase', 'band', 'work', 'phone', 'payment', 'recommend', 'case', 'delicate', 'phone', 'region', 'week', 'purchase', 'phone', 'seller', 'order', 'figment', 'work', 'week', 'gift', 'wrap', 'order', 'phone', 'seller', 'time', 'phone', 'family', 'delivery', 'week', 'quality', 'gift', 'seller', 'film', 'screen', 'trifle', 'nothing', 'gift', 'phone', 'fire', 'everything', 'seller', 'charger', 'c', 'type', 'everything', 'order', 'phone', 'book', 'seller', 'everything', 'day', 'quality', 'everything', 'day', 'phone', 'chelny', 'day', 'phone', 'everything', 'delivery', 'day', 'phone', 'impression', 'seller', 'moment', 'claim', 'phone', 'week', 'time', 'everything', 'seller', 'order', 'phone', 'work', 'hurray', 'order', 'time', 'phone', 'work', 'tyumen', 'week', 'everything', 'order', 'everything', 'day', 'phone', 'chelny']


In [10]:
from nltk import FreqDist
import nltk
#print(doc_clean)
#words = nltk.tokenize.word_tokenize(doc_clean)
all_reviews_in_one_string = " ".join([doc for doc in doc_clean])

words = nltk.tokenize.word_tokenize(all_reviews_in_one_string)
fdist = FreqDist(noun_phrases) 
frequent_words_freq = fdist.most_common(100)
print(frequent_words_freq )
#fdist.plot()
frequent_words= [w for (w,c) in frequent_words_freq]
#print(frequent_words)



[('phone', 4643), ('seller', 3124), ('day', 2265), ('everything', 1538), ('week', 1162), ('delivery', 1162), ('work', 1062), ('order', 1007), ('film', 808), ('time', 721), ('thank', 624), ('money', 621), ('gift', 524), ('quality', 459), ('recommend', 451), ('glass', 360), ('description', 360), ('problem', 358), ('case', 312), ('super', 309), ('track', 292), ('camera', 292), ('adapter', 290), ('screen', 289), ('purchase', 283), ('firmware', 283), ('region', 274), ('month', 274), ('product', 261), ('custom', 243), ('battery', 239), ('packaging', 225), ('price', 217), ('mail', 215), ('box', 205), ('\\\\_', 202), ('parcel', 189), ('thanks', 178), ('use', 169), ('smartphone', 168), ('russia', 164), ('advise', 161), ('shipping', 160), ('complaint', 148), ('store', 147), ('silicone', 139), ('post', 137), ('package', 137), ('fine', 135), ('dispute', 131), ('charge', 126), ('air', 122), ('cover', 121), ('way', 116), ('question', 111), ('extension', 110), ('photo', 109), ('star', 108), ('color',

def clean(doc):
    tokens = word_tokenize(doc)
    only_frqt_words = " ".join([token for token in tokens if token in frequent_words])
    return only_frqt_words 

#doc_clean2 = [clean(doc) for doc in doc_clean] 
doc_clean2 = [clean(doc).split() for doc in doc_clean] 
#print(doc_clean2)

Now find snippets in the review with these frequent nouns

In [11]:
def traverse(t,noun,phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            phrase = " ".join([word for (word,pos) in t.leaves()])
            if noun in phrase.split():
                phrases.append(phrase)
                #print(phrase)
        for child in t:
            traverse(child,noun,phrases)
    return phrases

from nltk.sentiment.vader import SentimentIntensityAnalyzer

dict_noun_abs_sentiment = {}
for noun in frequent_words:
    phrases=[]
    phrases=traverse(noun_phrase_tagged,noun,phrases)
    print(phrases[1:3])
    #doc_complete = nouns

    sid =  SentimentIntensityAnalyzer()
    sentiments = sum(abs(sid.polarity_scores(phrase)["compound"]) for phrase in phrases)/len(phrases)
    print(noun + " has absolute avg sentiment " + str(sentiments))
    dict_noun_abs_sentiment[noun]=sentiments

['smart phone recommend great phone', 'happy purchase band work perfect dial phone']
phone has absolute avg sentiment 0.40289711895910735
['happy purchase thank great phone seller', 'fine ecb 3 week put gift wrap order phone seller']
seller has absolute avg sentiment 0.4087893154190664
['everything tracked day packaging quality', 'excellent everything ok came 14 day']
day has absolute avg sentiment 0.2388339613970596
['fine seller forgot put adapter charger type c type everything', 'everything tracked day packaging quality']
everything has absolute avg sentiment 0.4027710273972607
['samara region two week', 'fine ecb 3 week put gift wrap order phone seller']
week has absolute avg sentiment 0.26532710526315806
['always super delivery 2 week quality', 'fine delivery 9 day']
delivery has absolute avg sentiment 0.3510705676855896
['store good good figment satisfied recommend work', 'model-the third work']
work has absolute avg sentiment 0.3732034383954153
['fine ecb 3 week put gift wrap or

['price class photo box already', 'дополнб review photo']
photo has absolute avg sentiment 0.3019247863247865
['three star jamb seller отправившего parcel mail russia', 'seller phone 5 star bravo']
star has absolute avg sentiment 0.24421376146788992
['already bag glass fact color black seller handsome', 'rose-gold beautiful color instead']
color has absolute avg sentiment 0.3353514563106794
['excellent everything ok came 14 day', 'seller-all excellent always phone']
excellent has absolute avg sentiment 0.6829231292517032
['write review', 'дополнб review photo']
review has absolute avg sentiment 0.28181727272727275
['phone safe sound', 'safe sound also fast gift wrap']
sound has absolute avg sentiment 0.461643571428571
['stavropol 12 day date order everything work fine firmware', 'fourth phone order aliekspress returned dispute 65 day date order seller']
date has absolute avg sentiment 0.25142680412371127
['loud conversation interlocutor hear clearly hand', 'well hand sealed securely']


In [19]:
import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
my_collection = mydb['features']
result = mydb.features.create_index([('feature', pymongo.ASCENDING)],unique=True)

In [20]:
features=  [ {"feature" : key} for (key,val) in dict_noun_abs_sentiment.items() if val >0.01]
print(features)

[{'feature': 'phone'}, {'feature': 'seller'}, {'feature': 'day'}, {'feature': 'everything'}, {'feature': 'week'}, {'feature': 'delivery'}, {'feature': 'work'}, {'feature': 'order'}, {'feature': 'film'}, {'feature': 'time'}, {'feature': 'thank'}, {'feature': 'money'}, {'feature': 'gift'}, {'feature': 'quality'}, {'feature': 'recommend'}, {'feature': 'glass'}, {'feature': 'description'}, {'feature': 'problem'}, {'feature': 'case'}, {'feature': 'super'}, {'feature': 'track'}, {'feature': 'camera'}, {'feature': 'adapter'}, {'feature': 'screen'}, {'feature': 'purchase'}, {'feature': 'firmware'}, {'feature': 'region'}, {'feature': 'month'}, {'feature': 'product'}, {'feature': 'custom'}, {'feature': 'battery'}, {'feature': 'packaging'}, {'feature': 'price'}, {'feature': 'mail'}, {'feature': 'box'}, {'feature': '\\\\_'}, {'feature': 'parcel'}, {'feature': 'thanks'}, {'feature': 'use'}, {'feature': 'smartphone'}, {'feature': 'russia'}, {'feature': 'advise'}, {'feature': 'shipping'}, {'feature':

In [21]:
my_collection.insert_many(features)